# Гипотеза о `font_face`

Предполагается следующее: изменение `font_face` учитывается добавлением константы.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

from lets_plot import *
LetsPlot.setup_html()

import os; import sys; sys.path.append(os.path.join(sys.path[0], ".."))
from util import util

In [2]:
def add_basic_width(df, basic_face):
    def calc_basic_width(r):
        basic_width = int(r[r.font_face == basic_face].iloc[0].width)
        r["basic_width"] = pd.Series(basic_width, index=r.index)
        return r

    df["basic_width"] = np.nan
    return df.groupby("char").apply(calc_basic_width)

In [3]:
BASIC_FONT_FACE = "normal"
BASIC_FONT = util.Font("Lucida Grande", 14, BASIC_FONT_FACE)

In [4]:
char_widths_df = util.get_df("../data/full/char_widths.csv", "all")
char_widths_df = util.filter_by_font(char_widths_df, BASIC_FONT, filters=["family", "size"])
char_widths_df = add_basic_width(char_widths_df, basic_face=BASIC_FONT_FACE)
char_widths_df["width_diff"] = char_widths_df.width - char_widths_df.basic_width
char_widths_df

,char_id,char,alphabet,font_face,width,basic_width,width_diff
0,65,A,basic_latin,normal,13,13,0
1,66,B,basic_latin,normal,13,13,0
2,67,C,basic_latin,normal,14,14,0
3,68,D,basic_latin,normal,14,14,0
4,69,E,basic_latin,normal,13,13,0
...,...,...,...,...,...,...,...
803,123,{,russian,bold+italic,9,6,3
804,124,|,russian,bold+italic,6,6,0
805,125,},russian,bold+italic,9,6,3
806,126,~,russian,bold+italic,11,11,0


In [5]:
font_faces = list(char_widths_df.font_face.unique())
font_faces

['normal', 'bold', 'italic', 'bold+italic']

## Шаг 1

В данных `'width_diff'` - разница между шириной символа для базового и текущего `font_face`.

**Гипотеза A** состоит в том, что величина `'width_diff'` постоянна для каждого `font_face`.

In [6]:
ggplot(char_widths_df) + \
    geom_violin(aes("font_face", "width_diff", fill="font_face"), kernel='rectangular') + \
    ggtitle("Вариация 'width_diff'")

Не похоже, что достаточно к каждому символу прибавить некоторую константу при переходе от одного `font_face` к другому. Кажется, разным символам нужны разные константы.

И все же, попробуем делать поправку с помощью только одного коэффициента (для каждого `font_face` - своего).

In [7]:
c_dict = char_widths_df.groupby("font_face").width_diff.mean().to_dict()
c_dict

{'bold': 0.6138613861386139,
 'bold+italic': 1.698019801980198,
 'italic': 1.0940594059405941,
 'normal': 0.0}

In [8]:
char_widths_df["width_predicted_by_c"] = char_widths_df.basic_width + char_widths_df.font_face.replace(c_dict)
char_widths_df["width_predicted_by_c_diff"] = char_widths_df.width - char_widths_df.width_predicted_by_c
char_widths_df.head()

,char_id,char,alphabet,font_face,width,basic_width,width_diff,width_predicted_by_c,width_predicted_by_c_diff
0,65,A,basic_latin,normal,13,13,0,13.0,0.0
1,66,B,basic_latin,normal,13,13,0,13.0,0.0
2,67,C,basic_latin,normal,14,14,0,14.0,0.0
3,68,D,basic_latin,normal,14,14,0,14.0,0.0
4,69,E,basic_latin,normal,13,13,0,13.0,0.0


In [9]:
ggplot(char_widths_df) + \
    geom_bar(aes(x="width_predicted_by_c_diff", fill="alphabet")) + \
    facet_grid(x="alphabet", y="font_face") + \
    xlab("ошибка предсказания") + ylab("количество символов") + ggtitle("График ошибок")

Судя по графику ошибок, **гипотеза A** не верна, и не достаточно просто взять и для каждого `'font_face'` прибавлять к ширине символа некоторое целое `c(F)`.

## Шаг 2

Попробуем еще раз посмотреть на графики.

In [10]:
ggplot(char_widths_df) + \
    geom_boxplot(aes("basic_width", "width_diff", fill="font_face")) + \
    facet_grid(x="font_face") + \
    ggtitle("Вариация 'width_diff' для разной 'basic_width'")

Из этого графика сложно увидеть какую-либо закономерность, стоит попробовать выделить среднюю линию.

In [11]:
ggplot(char_widths_df.groupby(["font_face", "basic_width"]).width_diff.mean().to_frame().reset_index()) + \
    geom_line(aes("basic_width", "width_diff", color="font_face")) + \
    facet_grid(x="font_face")

Судя по всему, можно найти некоторую функцию `f(x)`, такую, что если мы умеем предсказывать ширину `w(X, B)` символа `X` для `face=B`, то для `face=F` ширина должна получаться как `w(X, F) = w(X, B) + c(F) * f(w(X, B))`, где `c(F)` - некоторая константа, зависящая только от `F`. Это будет **гипотеза B**.

In [12]:
f_df = char_widths_df.groupby("basic_width").width_diff.mean().to_frame()
f_df.head()

,width_diff
basic_width,
4,1.700000
5,0.947368
6,0.657895
7,1.785714
8,3.375000


In [13]:
ggplot(f_df.reset_index()) + \
    geom_line(aes("basic_width", "width_diff")) + \
    ggtitle("f(x)")

In [14]:
char_widths_df["c"] = char_widths_df.width_diff / char_widths_df.basic_width.replace(f_df.to_dict()['width_diff'])
char_widths_df.head()

,char_id,char,alphabet,font_face,width,basic_width,width_diff,width_predicted_by_c,width_predicted_by_c_diff,c
0,65,A,basic_latin,normal,13,13,0,13.0,0.0,0.0
1,66,B,basic_latin,normal,13,13,0,13.0,0.0,0.0
2,67,C,basic_latin,normal,14,14,0,14.0,0.0,0.0
3,68,D,basic_latin,normal,14,14,0,14.0,0.0,0.0
4,69,E,basic_latin,normal,13,13,0,13.0,0.0,0.0


Согласно **гипотезе B** нам следует ожидать, что `c` - константа для каждого `'font_face'`.

In [15]:
ggplot(char_widths_df) + \
    geom_violin(aes("font_face", "c"))

Пока не похоже, что `c` константна для каждого `'font_face'`, но проверку с графиком ошибок мы все же сделаем.

In [16]:
c_dict = char_widths_df.groupby("font_face").c.mean().to_dict()
c_dict

{'bold': 0.6847655975187842,
 'bold+italic': 2.0526720740775244,
 'italic': 1.2625623284036915,
 'normal': 0.0}

In [17]:
char_widths_df["width_predicted_by_f"] = np.round(char_widths_df.basic_width + char_widths_df.font_face.replace(c_dict) * char_widths_df.basic_width.replace(f_df.to_dict()['width_diff'])).astype(int)
char_widths_df["width_predicted_by_f_diff"] = char_widths_df.width - char_widths_df.width_predicted_by_f
char_widths_df.head()

,char_id,char,alphabet,font_face,width,basic_width,width_diff,width_predicted_by_c,width_predicted_by_c_diff,c,width_predicted_by_f,width_predicted_by_f_diff
0,65,A,basic_latin,normal,13,13,0,13.0,0.0,0.0,13,0
1,66,B,basic_latin,normal,13,13,0,13.0,0.0,0.0,13,0
2,67,C,basic_latin,normal,14,14,0,14.0,0.0,0.0,14,0
3,68,D,basic_latin,normal,14,14,0,14.0,0.0,0.0,14,0
4,69,E,basic_latin,normal,13,13,0,13.0,0.0,0.0,13,0


In [18]:
ggplot(char_widths_df) + \
    geom_bar(aes(x="width_predicted_by_f_diff", fill="alphabet")) + \
    facet_grid(x="alphabet", y="font_face") + \
    xlab("ошибка предсказания") + ylab("количество символов") + ggtitle("График ошибок")

Как будто бы намного лучше не стало. Разве что распределение ошибок чуть больше похоже на нормальное (отдаленно) - в сравнении с предыдущей моделью.

## Шаг 3

И еще раз посмотрим на графики.

In [19]:
ggplot(char_widths_df) + \
    geom_qq2(aes("basic_width", "width", color="alphabet")) + \
    geom_qq2_line(aes("basic_width", "width", color="alphabet")) + \
    facet_grid(x="alphabet", y="font_face")

Вообще говоря, получается, что иногда сам тип распределения ширины по символам меняется при переходе от одного `'font_face'` к другому - об этом явно говорят некоторые из Q-Q plot'ов.

In [20]:
ggplot(char_widths_df) + \
    geom_point(aes("basic_width", "width", color="font_face"), alpha=.2) + \
    geom_smooth(aes("basic_width", "width", color="font_face"), deg=5) + \
    facet_grid(x="font_face") + \
    coord_fixed()

Видно, что от ошибки нельзя избавиться или сделать ее существенно меньше: есть много точек с разными `y` и одинаковыми `x`, что означает, что символы одной и той же ширины `w` меняются по разному при переходе к другому `'font_face'`. Лучшее на что можно рассчитывать - сделать ошибку распределенной максимально нормально для всех `'font_face'`, чтобы она, возможно, не слишком росла на текстах достаточно большого размера.

Другой вариант - отказаться от идеи прогнозировать изменение `'font_face'`, и воспринимать изменение `'font_face'` как замену одного шрифта на другой.

Последняя попытка нормализовать ошибку: предположить, что `w(X, F) = f_F(w(X, B))`, где `f_F(x)` - некоторый многочлен.

In [21]:
def append_prediction(df, name, deg=3):
    X = df[["basic_width"]]
    X_transformed = PolynomialFeatures(deg).fit_transform(X)
    y = df[["width"]]
    df[name] = np.round(LinearRegression().fit(X_transformed, y).predict(X_transformed).reshape(-1)).astype(int)
    return df

In [22]:
DEG = 5
char_widths_df = pd.concat([
    append_prediction(char_widths_df[char_widths_df.font_face == font_face].copy(), "width_predicted_by_reg", deg=DEG)
    for font_face in font_faces
])
char_widths_df["width_predicted_by_reg_diff"] = char_widths_df.width - char_widths_df.width_predicted_by_reg
char_widths_df.head()

,char_id,char,alphabet,font_face,width,basic_width,width_diff,width_predicted_by_c,width_predicted_by_c_diff,c,width_predicted_by_f,width_predicted_by_f_diff,width_predicted_by_reg,width_predicted_by_reg_diff
0,65,A,basic_latin,normal,13,13,0,13.0,0.0,0.0,13,0,13,0
1,66,B,basic_latin,normal,13,13,0,13.0,0.0,0.0,13,0,13,0
2,67,C,basic_latin,normal,14,14,0,14.0,0.0,0.0,14,0,14,0
3,68,D,basic_latin,normal,14,14,0,14.0,0.0,0.0,14,0,14,0
4,69,E,basic_latin,normal,13,13,0,13.0,0.0,0.0,13,0,13,0


In [23]:
ggplot(char_widths_df) + \
    geom_bar(aes(x="width_predicted_by_reg_diff", fill="alphabet")) + \
    facet_grid(x="alphabet", y="font_face") + \
    xlab("ошибка предсказания") + ylab("количество символов") + ggtitle("График ошибок")

Не такой уж плохой график ошибок получается при `DEG=5`. Но для `font_face='italic'` все равно ситуация чуть хуже, чем в остальных графиках. Может, есть смысл выделять в качестве отдельного шрифта хотя бы этот случай.